In [18]:
import cvxpy as cp 

 

# Constants 
weeks = range(1, 53)  # Weeks 1 to 52 
products = ["apples", "bananas", "oranges"] # Products

 

mu = {"apples": 1.85, "oranges": 1.46, "bananas": 0.60}  # Prices (USD)

costs = {"apples": 0.56, "oranges": 0.29, "bananas": 0.15}  # Costs (USD)

space_per_pound = {"apples": 0.04, "oranges": 0.03, "bananas": 0.05}  # Space per lb. of product (cubic ft)

base_demand = { # Base demand for product (lbs/wk)
    
    "apples": [2596] * 34 + [2856] * 11 + [2596] * 7, 
    
    "oranges": [865] * 5 + [952] * 41 + [865] * 6, 

    "bananas": [3750] * 52, 

} 

alpha = {"apples": 0.05, "oranges": 0.05, "bananas": 0.05} # Minimum percentage of display space for each product


s = 450  # Total space in cubic feet 

w = {"apples": 0.25, "oranges": 0.25, "bananas": 0.25} # Maximum percentage of weekly shipment for each product stored
 

# Elasticity functions 

def q(product, price): 
    
    adj_price = 0

    if product == "apples": 

        adj_price = 1 - 0.58 * ((price - mu["apples"]) / mu["apples"]) 

    elif product == "oranges": 

        adj_price = 1 - 1.10 * ((price - mu["oranges"]) / mu["oranges"]) 

    elif product == "bananas": 

        adj_price =  1 - 1.01 * ((price - mu["bananas"]) / mu["bananas"]) 
    
    # Demand cannot be negative
    if adj_price < 0: adj_price = 0
    
    return adj_price

 

# Decision variables 

x = { (i, j): cp.Variable(nonneg=True) for i in weeks for j in products } 

y = { (i, j): cp.Variable(nonneg=True) for i in weeks for j in products } 

 

# Objective function 

profit = cp.sum([x[i, j] * (mu[j] - costs[j]) for i in weeks for j in products]) 

objective = cp.Maximize(profit)
 

# Constraints 

constraints = [] 

 



for i in weeks: 

    for j in products: 

        # Demand satisfaction constraint 
        price_adjusted_demand = q(j, mu[j]) * base_demand[j][i - 1]
        
        prev_l = y[(i - 1, j)] if i > 1 else 0 
        
        constraints.append( 

            x[(i, j)] + prev_l - y[(i, j)] == price_adjusted_demand

        ) 
        
        # Minimum display percentage constraint
        constraints.append((x[(i, j)] * space_per_pound[j]) / s >= alpha[j]) 
        
        # Maximum storage percentage constraint
        constraints.append(y[(i, j)] <= w[j] * x[(i, j)])

 

# Space constraint 

for i in weeks: 

    constraints.append( 
        
        cp.sum([(x[(i, j)] + (y[(i - 1, j)] if i > 1 else 0) - y[(i, j)]) * space_per_pound[j] for j in products]) <= s

    ) 
    


# Boundary conditions for leftovers (none in week 52)

for j in products: 

    constraints.append(y[(52, j)] == 0) 

 

# Problem definition and solve 

problem = cp.Problem(objective, constraints) 

problem.solve(verbose=True) 

 

# Output 

if problem.status not in ["optimal", "optimal_inaccurate"]: 

    print(f"Problem status: {problem.status}") 

else: 

    for j in products: 

        print(f"Product: {j}") 

        for i in weeks: 

            print(f"Week {i}: Ordered = {x[(i, j)].value}, Stored = {y[(i, j)].value}") 

                                     CVXPY                                     
                                     v1.5.3                                    
(CVXPY) Nov 25 04:12:31 PM: Your problem has 312 variables, 523 constraints, and 0 parameters.
(CVXPY) Nov 25 04:12:31 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 25 04:12:31 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 25 04:12:31 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Nov 25 04:12:31 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Nov 25 04:12:31 PM: Compiling problem (target solver=CLARABEL